In [7]:
from process_and_ml.data_wrangler import CleanUp, load_azdias, load_customers, load_test_file
from process_and_ml.where_is import WhereIs

In [2]:

where_is = WhereIs(False)

In [3]:
all_paths = where_is.get_paths_list

In [8]:
#load azdias(population info)
azdias_df = load_azdias(all_paths[0])


/home/jair/.cache/pypoetry/virtualenvs/arvatocapstoneproject-L5ipTb8G-py3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3347: DtypeWarning: Columns (18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [9]:
customers_df = load_customers(all_paths[1])

/home/jair/.cache/pypoetry/virtualenvs/arvatocapstoneproject-L5ipTb8G-py3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3347: DtypeWarning: Columns (18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [ ]:
#Create dict for pipeline
dict_to_pipeline = {'unknowns_df':all_paths[4]}


In [4]:
clean_up = CleanUp(all_paths)



In [5]:
# Mount dict for pipeline, you can check on pipeline documentation
clean_azdias, clean_customer, clean_test = clean_up.pipeline_clean_up()

TypeError: pipeline_clean_up() missing 1 required positional argument: 'dfs'